## Dataset Diabetes

Este conjunto de datos proviene originalmente del Instituto Nacional de Diabetes y Enfermedades Digestivas y Renales. El objetivo del conjunto de datos es predecir si un paciente tiene o no diabetes, basándose en ciertas medidas diagnósticas incluidas en el conjunto de datos. Se impusieron varias restricciones en la selección de estas instancias de una base de datos más grande. En particular, todos los pacientes aquí son mujeres mayores de 21 años de herencia india Pima.

### Cargar los datos

Abrir el archivo `diabetes.csv` en la carpeta `1_datos`. Analizar su estructura y responder las siguientes preguntas:

*   ¿Cuántas instancias o ejemplos tiene el dataset?
*   ¿Cómo se llama la variable de salida o target? ¿Qué tipo de datos es?
*   ¿Cuántos atributos posee cada instancia o ejemplo?¿Puede detectar datos faltantes o mal registrados en algún atributo?
*   ¿Qué tipo de dato tiene cada atributo?

In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [34]:
# Cargamos los datos
df_diabetes = pd.read_csv("./1_datos/diabetes.csv")
X = df_diabetes.drop(columns="Outcome")

# Eliminar la primera columna que es una réplica del índice
df_diabetes.drop(columns="Unnamed: 0", inplace=True)
X = df_diabetes.drop(columns="Outcome")
Etiquetas = df_diabetes["Outcome"]

In [42]:
df_diabetes.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,763.000000,733.000000,541.000000,394.000000,757.000000,768.000000,768.000000,768.000000
mean,3.845052,121.686763,72.405184,29.153420,155.548223,32.457464,0.471876,33.240885,0.348958
std,3.369578,30.535641,12.382158,10.476982,118.775855,6.924988,0.331329,11.760232,0.476951
min,0.000000,44.000000,24.000000,7.000000,14.000000,18.200000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,64.000000,22.000000,76.250000,27.500000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,29.000000,125.000000,32.300000,0.372500,29.000000,0.000000
75%,6.000000,141.000000,80.000000,36.000000,190.000000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [35]:
# Número de instancias o ejemplos
num_instancias = df_diabetes.shape[0]

# Nombre de la variable de salida o target y su tipo de dato
variable_salida = 'Outcome'
tipo_dato_salida = df_diabetes[variable_salida].dtype

# Número de atributos por instancia
num_atributos = df_diabetes.shape[1] - 1  # Excluyendo la variable de salida

# Detección de datos faltantes o mal registrados
datos_faltantes = df_diabetes.isnull().sum()

# Tipo de dato de cada atributo
tipos_datos = df_diabetes.dtypes

# Imprimir resultados
print(f"Número de instancias: {num_instancias}")
print(f"Variable de salida: {variable_salida}, Tipo de dato: {tipo_dato_salida}")
print(f"Número de atributos por instancia: {num_atributos}")
print("Datos faltantes por atributo:")
print(datos_faltantes)
print("Tipo de dato de cada atributo:")
print(tipos_datos)

Número de instancias: 768
Variable de salida: Outcome, Tipo de dato: int64
Número de atributos por instancia: 8
Datos faltantes por atributo:
Pregnancies                  0
Glucose                      0
BloodPressure                1
SkinThickness               12
Insulin                      5
BMI                          1
DiabetesPedigreeFunction     0
Age                          0
Outcome                      0
dtype: int64
Tipo de dato de cada atributo:
Pregnancies                   int64
Glucose                       int64
BloodPressure               float64
SkinThickness               float64
Insulin                     float64
BMI                         float64
DiabetesPedigreeFunction    float64
Age                           int64
Outcome                       int64
dtype: object


### ⚠️
Antes de realizar cualquier procesamiento, no olvide de separar los datos en entrenamiento y prueba. Divida el conjunto de datos en entrenamiento y prueba utilizando train_test_split (con una proporción de 80% entrenamiento y 20% prueba) use 42 como semilla.

In [36]:
from sklearn.model_selection import train_test_split

# division de los datos en test y prueba mas las etiquetas
Predictoras_train, Predictoras_test, Etiquetas_train, Etiquetas_test = train_test_split(X, Etiquetas, test_size=0.2, shuffle=True, random_state=42, stratify=Etiquetas)

### Detección de valores atípicos y tratamiento de datos faltantes

- Analice con cuidado el rango de valores de los atributos. **¿Nota valores atípicos a simple vista en algún/algunos de los atributos?**

    -  Ejemplo: Un valor de insulina igual a 0 puede considerarse mal registrado o faltante.

- Los valores atípicos que detecte pueden considerarse datos faltantes. Utilice el método [replace](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.replace.html) de pandas para reemplazarlos por np.NaN.

In [37]:
# Identificar valores atípicos y reemplazarlos por np.NaN
valores_atipicos = {
    'Glucose': 0,
    'BloodPressure': 0,
    'SkinThickness': 0,
    'Insulin': 0,
    'BMI': 0
}

df_diabetes.replace(valores_atipicos, np.NaN, inplace=True)

Después de realizar el reemplazo, ¿Cuántos datos faltantes presenta cada atributo?

In [38]:
# Verificar los datos faltantes después del reemplazo
datos_faltantes_actualizados = df_diabetes.isnull().sum()
print("Datos faltantes por atributo después del reemplazo de valores atípicos:")
print(datos_faltantes_actualizados)

Datos faltantes por atributo después del reemplazo de valores atípicos:
Pregnancies                   0
Glucose                       5
BloodPressure                35
SkinThickness               227
Insulin                     374
BMI                          11
DiabetesPedigreeFunction      0
Age                           0
Outcome                       0
dtype: int64


### Aclaración sobre la imputación de datos faltantes
Algo a tener en cuenta al momento de imputar datos faltantes es que los atributos pueden tener distribuciones diferentes según la salida o target (con diabetes/sin diabetes). Por ejemplo, una persona sana tendrá un valor de insulina en un rango distinto a una persona con diabetes.

- Implemente un transformer para imputar datos faltantes de forma condicional, dependiendo de la clase (diabetes/no diabetes). Para hacerlo, utilice la media o mediana correspondiente a cada clase.
- Utilice un Pipeline de scikit-learn para realizar la preparación completa de los datos, que incluya la imputación.

In [41]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline

class ConditionalImputer(BaseEstimator, TransformerMixin):
    def __init__(self, strategy='mean'):
        self.strategy = strategy
        self.fill_values_ = {}

    def fit(self, X, y=None):
        if y is None:
            raise ValueError("y cannot be None")

        self.fill_values_ = {}
        for column in X.columns:
            self.fill_values_[column] = {}
            for cls in y.unique():
                if self.strategy == 'mean':
                    self.fill_values_[column][cls] = X.loc[y == cls, column].mean()
                elif self.strategy == 'median':
                    self.fill_values_[column][cls] = X.loc[y == cls, column].median()
                else:
                    raise ValueError("Strategy not supported")
        return self

    def transform(self, X):
        X_transformed = X.copy()
        for column in X.columns:
            for cls, value in self.fill_values_[column].items():
                X_transformed.loc[X_transformed[column].isnull() & (Etiquetas == cls), column] = value
        return X_transformed

# Crear el pipeline
pipeline = Pipeline(steps=[
    ('conditional_imputer', ConditionalImputer(strategy='mean'))
])

# Ajustar el pipeline a los datos de entrenamiento
pipeline.fit(df_diabetes, Etiquetas)

# Transformar los datos de entrenamiento y prueba
Predictoras_train_imputed = pipeline.transform(df_diabetes)
Predictoras_test_imputed = pipeline.transform(df_diabetes)

C:\Users\yeyer\AppData\Local\Temp\ipykernel_27220\3452857813.py:29: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '4.865671641791045' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  X_transformed.loc[X_transformed[column].isnull() & (Etiquetas == cls), column] = value
C:\Users\yeyer\AppData\Local\Temp\ipykernel_27220\3452857813.py:29: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '37.06716417910448' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  X_transformed.loc[X_transformed[column].isnull() & (Etiquetas == cls), column] = value
C:\Users\yeyer\AppData\Local\Temp\ipykernel_27220\3452857813.py:29: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '4.865671641791045' 


- Entrene al menos tres clasificadores distintos.
- Compare el desempeño de los modelos utilizando la métrica F1-score, ya que el dataset tiene un desbalance de clases (más casos sin diabetes que con diabetes). Puede evaluar otras métricas si lo desea. [sklearn metrics](https://scikit-learn.org/stable/modules/model_evaluation.html)